In [1]:
import mediapipe as mp

In [2]:
import cv2 as cv
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

In [3]:
detector=FaceMeshDetector(maxFaces=1)

In [4]:
cap=cv.VideoCapture(0)
while True:
    _,frame=cap.read()
    img,faces=detector.findFaceMesh(frame,draw=False)
    if faces:
        face=faces[0]
        left_eye,right_eye=face[145],face[374]
        cv.line(img,left_eye,right_eye,(0,255,0),3,cv.LINE_AA)
        cv.circle(img,left_eye,3,(0,0,255),cv.FILLED)
        cv.circle(img,right_eye,3,(0,0,255),cv.FILLED)

        dist_in_pixels,_=detector.findDistance(left_eye,right_eye)
        average_human_eye_dist=6.3  # in cms
        focal_lenght=840
        # finding the d which is distance from object to focal point
        d=(average_human_eye_dist*focal_lenght)/dist_in_pixels
        cv.putText(img,f"{d} in cm",(200,400),cv.FONT_HERSHEY_COMPLEX,1,(0,0,0),2,cv.LINE_AA)


    
    if cv.waitKey(1)==27:
        cap.release()
        cv.destroyAllWindows()
        break

    cv.imshow('frame',img)

In [10]:
faces[0]

[[278, 323],
 [279, 302],
 [279, 307],
 [273, 275],
 [279, 295],
 [279, 284],
 [280, 257],
 [229, 246],
 [280, 239],
 [281, 230],
 [282, 190],
 [278, 327],
 [278, 329],
 [278, 329],
 [278, 330],
 [278, 334],
 [278, 337],
 [278, 341],
 [278, 347],
 [279, 305],
 [272, 303],
 [203, 213],
 [250, 254],
 [243, 254],
 [236, 254],
 [227, 249],
 [256, 252],
 [239, 238],
 [247, 239],
 [232, 239],
 [228, 240],
 [221, 253],
 [251, 355],
 [226, 244],
 [199, 244],
 [212, 247],
 [244, 287],
 [270, 322],
 [271, 328],
 [261, 322],
 [257, 324],
 [265, 327],
 [261, 327],
 [248, 332],
 [273, 301],
 [272, 295],
 [216, 232],
 [258, 269],
 [256, 295],
 [256, 290],
 [224, 284],
 [273, 284],
 [233, 228],
 [223, 228],
 [209, 203],
 [265, 237],
 [253, 241],
 [242, 326],
 [205, 309],
 [261, 299],
 [267, 302],
 [251, 327],
 [254, 326],
 [219, 223],
 [256, 297],
 [246, 230],
 [245, 224],
 [238, 191],
 [214, 213],
 [242, 207],
 [212, 227],
 [207, 221],
 [270, 326],
 [264, 326],
 [259, 326],
 [263, 300],
 [253, 327],

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from mediapipe import solutions as mp
import cv2 as cv
import numpy as np

In [7]:
mp_drawing=mp.drawing_utils
mp_holistic=mp.holistic

In [8]:
holistic=mp_holistic.Holistic(min_detection_confidence=0.7)

In [9]:
def extract_landmarks(results):
    lh=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    pose=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    return np.concatenate([lh,rh,pose])

In [10]:
import pickle
ready_model=pickle.load(open('ready/ready_model.pickle','rb'))

In [11]:
VAL_1=False  #user in range
VAL_2=False  #user is ready
COMPARE_1=179-3
COMPARE_2=192
d=0

In [12]:
start_point1=(180+20,0)
end_point1=(180+20,480)
start_point2=(440,0)
end_point2=(440,480)

In [18]:
cap=cv.VideoCapture(0)
while True:
    _,frame=cap.read()
    image=cv.cvtColor(frame,cv.COLOR_BGR2RGB)
    results=holistic.process(image)
    # image=cv.cvtColor(image,cv.COLOR_RGB2BGR)
    # draw_landmarks(results,image)
    landmarks=extract_landmarks(results)
    
    img,faces=detector.findFaceMesh(frame,draw=False)
    

    cv.line(img,start_point1,end_point1,(0,0,255),3)
    cv.line(img,start_point2,end_point2,(0,0,255),3)
    
    if faces:
        face=faces[0]
        left_eye,right_eye=face[145],face[374]
        cv.line(img,left_eye,right_eye,(0,255,0),3,cv.LINE_AA)
        cv.circle(img,left_eye,3,(0,0,255),cv.FILLED)
        cv.circle(img,right_eye,3,(0,0,255),cv.FILLED)

        dist_in_pixels,_=detector.findDistance(left_eye,right_eye)
        average_human_eye_dist=6.3  # in cms
        focal_lenght=840
        # finding the d which is distance from object to focal point
        d=int((average_human_eye_dist*focal_lenght)/dist_in_pixels)
        # cv.putText(img,f"{d} in cm",(200,400),cv.FONT_HERSHEY_COMPLEX,1,(0,0,0),2,cv.LINE_AA)
        
    # if user in range of predictions
    if COMPARE_1 <= d <=COMPARE_2:
        VAL_1=True
        cv.line(img,start_point1,end_point1,(0,255,0),3)
        cv.line(img,start_point2,end_point2,(0,255,0),3)
    else:
        VAL_1=False
        VAL_2=False

    if VAL_1:
        text=ready_model.predict(np.expand_dims(landmarks,0))[0]
        cv.putText(img,text,(400,400),cv.FONT_HERSHEY_COMPLEX_SMALL,3,(255,0,0),1,cv.LINE_AA)
        if text=='ready':
            VAL_2=True
            
    if VAL_1 and VAL_2:
        cap.release()
        cv.destroyAllWindows()
        break
    
    if cv.waitKey(1)==27:
        cap.release()
        cv.destroyAllWindows()
        break

    cv.imshow('frame',img)

In [44]:
cap.release()

In [ ]:
cap=cv.VideoCapture(0)
i=5
while cap.isOpened():
    _,frame=cap.read()
    display=i
    if i==5:
        display='Get Started'
        cv.putText(frame,display,(30,200),cv.FONT_HERSHEY_COMPLEX,2,(0,0,255),1,cv.LINE_AA)
    else:
        cv.putText(frame,str(display),(30,200),cv.FONT_HERSHEY_COMPLEX,2,(0,0,255),1,cv.LINE_AA)
    i-=1    

    if i==0:
        cap.release()
        cv.destroyAllWindows()
        break
    
    if cv.waitKey(100000)==27:
        cap.release()
        cv.destroyAllWindows()
        break
    
    
    cv.imshow('frame',frame)

In [ ]:
cap.release()

In [41]:
cap=cv.VideoCapture(0)
i=0
while True:
    _,frame=cap.read()
    
    if 0<i<70:
        cv.putText(frame,'Getting Started',(100,250),cv.FONT_HERSHEY_DUPLEX,2,(0,255,0),2,cv.LINE_AA)
    elif 70<i<140:
        cv.putText(frame,'3',(260,250),cv.FONT_HERSHEY_DUPLEX,4,(0,0,255),2,cv.LINE_AA)
    elif 140<i<190:
        cv.putText(frame,'2',(260,250),cv.FONT_HERSHEY_DUPLEX,4,(0,0,255),2,cv.LINE_AA)
    elif 190<i<240:
        cv.putText(frame,'1',(260,250),cv.FONT_HERSHEY_DUPLEX,4,(0,0,255),2,cv.LINE_AA)
    elif 240<i<300:
        cv.putText(frame,'GO',(260,250),cv.FONT_HERSHEY_DUPLEX,4,(0,255,0),2,cv.LINE_AA)
    i+=1
    if i==300:
        cap.release()
        cv.destroyAllWindows()
        break
    if cv.waitKey(10)==27:
        cap.release()
        cv.destroyAllWindows()
        break

    cv.imshow('frame',frame)

In [24]:
cap.release()

In [22]:
i

259

In [11]:
from datetime import datetime

In [13]:
t1=datetime.now()

In [14]:
t2=datetime.now()

In [16]:
t=t1-t2

In [20]:
t.seconds

86391